In [13]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
plt.close('all')
matplotlib.use('TkAgg')

import scipy.optimize as opt
import scipy

import xlwings as xw
import os
import seaborn as sns

n = 10 # number of vials

# Efficiency at low activities (<0.05 $\mu$Ci)

Looking to investigate the efficiency of the Hidex at low activities, so need to reduce the noise in the measurements. To do this:
1. Find an appropriate activity (low noise) to measure the FWHM of both peaks (511 keV and 1022 keV)
2. Reprocess all data from set 4 that is <0.1 $\mu$Ci and extract counts for both peaks +- FWHM
3. Pass the above counts to the efficiency script

## 1. Find the FWHM of both peaks

In [14]:
# get the data from Excel (NOTE: the original files aren't well-formatted so i had to extract 
# the spectra by hand. However for step 2 I will need to work directly with the Excel files so this is a problem for later.)

f = open("./data/HidexAMG-Track_30min-035-20240502-013749-AutoExport_SPECTRA.xlsx", "rb")
sample_spectra = pd.read_excel(f,
                               engine='openpyxl')
f.close()

sample_spectra = sample_spectra.to_numpy()[:, 2:] #ignores the rack and vial numbers

In [15]:
def gauss(x, p): # p[0]==mean, p[1]==stdev, p[2]==normalization factor
    return p[2]*np.exp(-(x-p[0])**2/(2*p[1]**2))

FWHM_511_list = []
FWHM_1022_list = []

for i in range(0,10):
    plt.plot(sample_spectra[i, :])

    X = np.linspace(0, 2047, 2048)
    Y = sample_spectra[i, :]
    
    # Fit a guassian to the 511 keV peak
    p0 = [511,1, 2000] # Inital guess is a normal distribution
    errfunc = lambda p, x, y: gauss(x, p) - y # Distance to the target function
    
    p1, success = opt.leastsq(errfunc, p0[:], args=(X, Y))
    
    fit_mu, fit_stdev, fit_height = p1
    
    FWHM = 2*np.sqrt(2*np.log(2))*fit_stdev
    FWHM_511_list.append(FWHM)
    plt.plot(gauss(X, p1))

    # Fit a guassian to the 1022 peak
    p0 = [1022,1, 2000] # Inital guess is a normal distribution
    errfunc = lambda p, x, y: gauss(x, p) - y # Distance to the target function
    
    p1, success = opt.leastsq(errfunc, p0[:], args=(X, Y))
    
    fit_mu, fit_stdev, fit_height = p1
    
    FWHM = 2*np.sqrt(2*np.log(2))*fit_stdev
    FWHM_1022_list.append(FWHM)
    plt.plot(gauss(X, p1))

plt.xlabel("Channel (keV)")
plt.ylabel("Counts")
plt.grid()
plt.show()

In [9]:
FWHM_1022 = np.mean(FWHM_1022_list)
s_FWHM_1022 = np.std(FWHM_1022_list)

FWHM_511 = np.mean(FWHM_511_list)
s_FWHM_511 = np.std(FWHM_511_list)

print(FWHM_1022, "+-", s_FWHM_1022)
print(FWHM_511, "+-", s_FWHM_511)

54.11739367081469 +- 0.6751569326997817
39.3216609300854 +- 0.2736845499617138


## 2. Generate the counts of the low-activity data with the new windows
1. pick files with low enough activity and move them to the correct subfolder
2. use python to find the spectra in these files, then add up the counts in the correct windows (also calculate the 400-600 window to compare)

In [10]:
# start with one file then i'll make it into a function
# NOT THE FINAL VERSION (SEE BELOW)
filepath = r"C:\Users\yuliy\OneDrive\Documents\Fall 2023\Research\efficiency\low_activity\data\low_activity_files\HidexAMG-Track_30min-035-20240502-110624-AutoExport.xlsx"

ws = xw.Book(filepath).sheets['Spectra']

spectra = ws["C20"].options(np.array, expand='table').value
p1_start, p1_end, p2_start, p2_end = int(511 - FWHM_511), int(511 + FWHM_511), int(1022 - FWHM_1022), int(1022 + FWHM_1022) #peak edges
counts = np.sum(spectra[:, p1_start:p1_end+1], axis=1) + np.sum(spectra[:, p2_start:p2_end+1], axis=1) # add up the counts in both peaks
counts_old = np.sum(spectra[:, 400:601], axis=1) # sanity check, these are the same as the old data

print(p1_start, p1_end, p2_start, p2_end)

471 550 967 1076


### 2.1 Remove background spectra
The background counts are not the same in every energy channel. We want to obtain a less noisy background spectrum from 14 background files taken at different times of the day, that we can subtract from the measured spectra from the samples.

In [11]:
# note: not using data file HidexAMG-Background Long-016-20230608-121915-AutoExport because it is unusually flat compared to the other ones
bg_path = "./data/backgrounds/"
for _, _, bg_files in os.walk(bg_path): break

bg_data = np.zeros((len(bg_files), 2048))

excel_app = xw.App(visible=False)

i=0
for file in bg_files:
    if "~" in file: continue # disregards temporary files
    path = bg_path + file
    
    # open up the excel sheet with the spectrum
    wb = xw.Book(path)
    sheet = wb.sheets['Spectra']
    spectra = sheet["C20"].options(np.array, expand='table').value
    wb.save()
    wb.close()
    
    bg_data[i, :] = spectra/30 # save it in CPM
    
    plt.plot(spectra/30, "grey")

    #print(np.sum(spectra[400:601])/30) # sanity check: should be around 42
    i+=1

excel_app.quit()

bg_spectrum = np.mean(bg_data, axis=0)
plt.title("Background averaging")
plt.ylabel("CPM")
plt.plot(bg_spectrum, "r")
plt.show()

np.savetxt('background.txt', bg_spectrum)

### 2.2 Calculate the counts

In [319]:
def counts(file, peak_options="20-10", remove_background=True):
    # argument is the path to the excel file generated by the hidex. There should be a sheet titled Spectra
    # the FWHM are defined above
    # peak_options = "FWHM", "2FWHM" or "20-10"
    path = "./data/low_activity_files/" + file
    wb = xw.Book(path)
    sheet = wb.sheets['Spectra']
    spectra = sheet["C20"].options(np.array, expand='table').value
    wb.save()
    wb.close()

    # remove background calculated above
    if remove_background:
        spectra -= bg_spectrum

    if peak_options == "2FWHM":
        p1_start, p1_end, p2_start, p2_end = int(511 - FWHM_511), int(511 + FWHM_511), int(1022 - FWHM_1022), int(1022 + FWHM_1022) #peak edges
    elif peak_options == "20-10":
        p1_start, p1_end, p2_start, p2_end = 409, 613, 920, 1124 # 20% around main peak, 10% around coincidence peak

    elif peak_options == "optimal":
        p1_start, p1_end, p2_start, p2_end =  389, 634, 1007, 1038 # as calculated below
    elif peak_options == "1FWHM":
        p1_start, p1_end, p2_start, p2_end = int(511 - FWHM_511/2), int(511 + FWHM_511/2), int(1022 - FWHM_1022/2), int(1022 + FWHM_1022/2)
    else: return
    
    counts = np.sum(spectra[:, p1_start:p1_end+1], axis=1) + np.sum(spectra[:, p2_start:p2_end+1], axis=1) # add up the counts in both peaks
    counts_old = np.sum(spectra[:, 400:601], axis=1) # sanity check, these are the same as the old data

    # also collect the timestamps so that we don't get lost in the sauce
    wb = xw.Book(path)
    ws = wb.sheets['Results']
    timestamps = ws["C27:C36"].options(np.array, dtype='datetime64').value
    wb.save()
    wb.close()
    
    return counts, counts_old, timestamps

# walk through the folder with the files and collect the counts for all of them
for root, dirs, files in os.walk("./data/low_activity_files/"): break

In [320]:
excel_app = xw.App(visible=False)

counts_list = []
counts_2fwhm_list = []
counts_1fwhm_list = []
old_counts_list = []
timestamps_list = []
counts_opt_list= []

for f in files:
    if "~" in f: continue # disregards temporary files
    print(f)
    
    c, c_old, t = counts(f)
    c_2fwhm, _, _ = counts(f, peak_options="2FWHM") 
    c_fwhm, _, _ = counts(f, peak_options="1FWHM") 
    c_opt, _, _ = counts(f, peak_options="optimal") 
    # TODO: implement this directly into the function since opening/closing excel files takes a long time
    
    counts_list.extend(c.tolist())
    counts_opt_list.extend(c_opt.tolist())
    counts_2fwhm_list.extend(c_2fwhm.tolist())
    counts_1fwhm_list.extend(c_fwhm.tolist())
    timestamps_list.extend(t)
    old_counts_list.extend(c_old.tolist())

print("Done.")
excel_app.quit()

HidexAMG-Track_30min-035-20240502-082900-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-084445-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-090029-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-091613-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-093157-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-094742-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-100326-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-101910-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-103454-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-105040-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-110624-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-112209-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-113753-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-115338-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-120922-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-122509-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-124055-AutoExport.xlsx
HidexAMG-Track_30min-035-202405

## 3. Checking if the noise is reduced
Finally, we want to compare if there is a difference in the noise between the two methods (400-600 window or two peaks windows)

In [321]:
# Sanity check: plot the old vs the new counts
plt.plot(timestamps_list, old_counts_list, ".", label="400-600 keV window")
plt.plot(timestamps_list, counts_list, ".", label="Two peaks, 20%-10% windows")
plt.plot(timestamps_list, counts_2fwhm_list, ".", label="Two peaks, 2 FWHM")
plt.ylabel("CPM")
plt.xlabel("timestamp")
plt.yscale("log")
plt.grid()
plt.legend()
plt.show()

In [325]:
# pablo wants to fit linear functions to the log-scaled data, and compare the r-squared
# need to separate individual vials for this
timestamps = np.array(timestamps_list).astype('datetime64[s]').astype('int') - timestamps_list[0].astype('datetime64[s]').astype('int')



r2_old = []
r2_2fwhm = []
r2_2010 = []
r2_1fwhm = []
r2_opt = []

for i in range(0, n):
    x = timestamps[i::n]
    y = np.log(old_counts_list[i::n])
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    print(r_value**2)
    r2_old.append(r_value**2)

print()

for i in range(0, n):
    x = timestamps[i::n]
    y = np.log(counts_opt_list[i::n])
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    print(r_value**2)
    r2_opt.append(r_value**2)

print()
for i in range(0, n):
    x = timestamps[i::n]
    y = np.log(counts_2fwhm_list[i::n])
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    print(r_value**2)
    r2_2fwhm.append(r_value**2)

print()
for i in range(0, n):
    x = timestamps[i::n]
    y = np.log(counts_list[i::n])
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    print(r_value**2)
    r2_2010.append(r_value**2)

print()
for i in range(0, n):
    x = timestamps[i::n]
    y = np.log(counts_1fwhm_list[i::n])
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    print(r_value**2)
    r2_1fwhm.append(r_value**2)

0.9954611677827839
0.997678239500497
0.9981095800029982
0.9952707223516475
0.9979671228599021
0.9989345872345392
0.999049690777638
0.9985008623730792
0.9987502370039555
0.9993449942982765

0.9971679156133104
0.9981073694875202
0.9980114713257681
0.9963210124163535
0.9977682170101103
0.9989982556313933
0.9990199949928253
0.9986117173310716
0.9990412088509322
0.9994437243768037

0.9953670331986866
0.9985604568193478
0.9980906651670708
0.9946405768823385
0.9977633266383663
0.9987994885403814
0.9991527922538346
0.9985236420245028
0.999348669248496
0.9994166436597706

0.9951174511504445
0.9980593999424948
0.998522092418439
0.9957142870126072
0.9972891030799208
0.998911716236857
0.9989831360658483
0.9987166396217567
0.9992449589465527
0.9995598094486665

0.9949088557729504
0.9977276450148542
0.9968626795264035
0.9941916428060016
0.9970700262073903
0.9973555554941482
0.9985189910710028
0.998511278477066
0.9993211558100767
0.9989878777279052


In [330]:
vials = np.linspace(1, 10, 10)
plt.plot(vials, r2_old, ".", label="400-600 keV window")
#plt.plot(vials, r2_2fwhm, ".", label="Two peaks, 2 FWHM")
#plt.plot(vials, r2_1fwhm, ".", label="Two peaks, 1 FWHM")
plt.plot(vials, r2_2010, ".", label="Two peaks, 20%-10% windows")
plt.plot(vials, r2_opt, "k+", label="Optimal")
plt.legend()
plt.xlabel("Vial")
plt.ylabel("R$^2$ value of linear fit")
plt.grid()
plt.show()

print("400-600 keV window:", np.mean(r2_old))
print("Two peaks, 2 FWHM:", np.mean(r2_2fwhm))
print("Two peaks, 1 FWHM:", np.mean(r2_1fwhm))
print("Two peaks, 20%-10% windows:", np.mean(r2_2010))
print("Best:", np.mean(r2_opt))

400-600 keV window: 0.9979067204185317
Two peaks, 2 FWHM: 0.9979663294432795
Two peaks, 1 FWHM: 0.9973455707907799
Two peaks, 20%-10% windows: 0.9980118593923588
Best: 0.998249088703609


# Optimizing like grownups
In this section, I want to properly optimize the energy windows of both peaks such that the r$^2$ value is as close to 1 as possible. 

## 4.1 Extract spectra from all files and put them in one file/array
Also the timestamps at the same time.

In [265]:
# walk through the folder with the files and collect the spectra
for _, _, files in os.walk("./data/low_activity_files/"): break
files = list(filter(lambda k: '~' not in k, files)) # remove temporary files to ensure size allocation is correct for spectra and timestamps

excel_app = xw.App(visible=False)

spectra = np.empty(shape=(len(files)*n, 2048)) # size: number of total runs (files x vials) by number of energy channels
spectra.fill(np.nan)

timestamps = np.empty(shape=(len(files)*n))
timestamps.fill(np.nan)

i = 0
for f in files:
    #if "~" in f: continue # disregards temporary files
    print(f)
    
    # extract the correct spectra from the file
    path = "./data/low_activity_files/" + f
    wb = xw.Book(path)
    sheet = wb.sheets['Spectra']
    s = sheet["C20"].options(np.array, expand='table').value
    
    # save the spectra to the array
    spectra[i*n:(i*n)+n, :] = s

    # extract the timestamps
    ws = wb.sheets['Results']
    t = ws["C27:C36"].options(np.array, dtype='datetime64').value

    # save the timestamps to an array
    timestamps[i*n:(i*n)+n] = t
    
    wb.save()
    wb.close()

    
    i += 1

print("Done.")
print("Sanity check: Any NaNs in the spectra array?", np.isnan(np.sum(spectra)))
print("Sanity check: Any NaNs in the timestamps array?", np.isnan(np.sum(timestamps)))
excel_app.quit()

# remove background from all spectra
spectra -= bg_spectrum

# start the clock at 0
timestamps -= timestamps[0]

HidexAMG-Track_30min-035-20240502-082900-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-084445-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-090029-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-091613-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-093157-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-094742-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-100326-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-101910-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-103454-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-105040-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-110624-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-112209-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-113753-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-115338-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-120922-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-122509-AutoExport.xlsx
HidexAMG-Track_30min-035-20240502-124055-AutoExport.xlsx
HidexAMG-Track_30min-035-202405

## 4.2 Visual sanity check: calculate r$^2$ with respect to various window widths
Steps:
1. Make a mesh of window widths (for the 1st and 2nd peak)
2. Calculate the corresponding energy channel bounds
3. Using the bounds calculated above, pass them to the counts function 

In [302]:
# Peak half-widths (K1 is the 511 peak, K2 is the 1022 peak)
K1 = (np.arange(180, 281, 2)).astype('int32')
K2 = (np.arange(20, 121, 2)).astype('int32')

In [303]:
def counts_custom_widths(spect, first_peak_widths, second_peak_widths):
    # a different implementation where the input is a file of all the spectra we want. this should already be background-subtracted
    # shape: n runs x 2048 channels
    
    # first_peak_widths: np array containing the widths of the 511 kev peak
    # second_peak_widths: np array containing the widths of the 1022 kev peak

    # this function meant for quicker calculation of the counts for various peak widths
    
    first_peak_widths = (first_peak_widths/2).astype('int32') # calculate half-widths
    second_peak_widths = (second_peak_widths/2).astype('int32')

    counts = np.empty(shape=(len(first_peak_widths), len(second_peak_widths), np.shape(spect)[0])) # shape: size of K1 x size of K2 x n runs
    counts.fill(np.nan)

    for i, W1 in enumerate(first_peak_widths):
        for j, W2 in enumerate(second_peak_widths):
            ## 1. calculate the peak edges for all desired widths
            p1_start, p1_end, p2_start, p2_end = 511 - W1, 511 + W1, 1022 - W2, 1022 + W2

            ## 2. calculate the corresponding counts
            c = np.sum(spect[:, p1_start:p1_end+1], axis=1) + np.sum(spectra[:, p2_start:p2_end+1], axis=1) # add up the counts in both peaks
            counts[i, j, :] = c
            
    
    counts_old = np.sum(spect[:, 400:601], axis=1) # sanity check, these are the same as the old data
    print("Sanity check: Any NaNs in the counts array?", np.isnan(np.sum(counts)))
    return counts, counts_old

r, _ = counts_custom_widths(spectra, K1, K2) # r has shape: lenght of K1 x length of K2 x number of data runs (310)

Sanity check: Any NaNs in the counts array? False


In [304]:
# We have the counts for a range of peak widths for both peaks. Time to find the R2 values for each vial and average them
r_squared_full = np.empty(shape=(n, len(K1), len(K2)))
r_squared_full.fill(np.nan)

for i in range(0, n):
    for j, W1 in enumerate(K1):
        for k, W2 in enumerate(K2):
            x = timestamps[i::n]
            y = np.log(r[j, k, i::n])

            slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
            r_squared_full[i,j,k] = r_value**2

r_squared = np.mean(r_squared_full, axis=0)

In [12]:
# find minimum of r2
k2_min = np.where(r_squared == r_squared.max())[0]
k1_min = np.where(r_squared == r_squared.max())[1]

ax = sns.heatmap(r_squared, xticklabels=K2, yticklabels=K1)
ax.set(xlabel="1022 keV peak width", ylabel="511 keV peak width", title="Optimising r$^2$ based on energy window widths")
plt.plot(k1_min, k2_min, "k+")
plt.show()

NameError: name 'r_squared' is not defined

It looks like the best windows are K1 = 245 keV and K2 = 31 keV